In [ ]:
print('Importing libraries')
%run setup.ipynb
print('Importing plotting rules and functions')
import plotting_functions
print('Importing time series functions')
from tseries_functions import *
from tide_functions import calculate_ntr

In [ ]:
ds = xr.open_dataset(data_dir / 'rsl_hawaii_noaa.nc')

# change all the station_ids to integers
ds['station_id'] = ds['station_id'].astype(int)


# get the ntr for each station
calculate_ntr(ds)

In [ ]:
# load ntr data
station_ids = ds['station_id'].values

station_id = station_ids[5]
print('Doing station', station_id)
station_name = ds.station_name.sel(station_id=station_id).item()
print('Station name:', station_name)
mhhw = ds.MHHW.sel(station_id=station_id).item()
msl = ds.MSL.sel(station_id=station_id).item()
mllw = ds.MLLW.sel(station_id=station_id).item()
ntrpath = f'ntr_data/ntr_{station_id}.csv'


ntr_data = pd.read_csv(Path(data_dir / ntrpath), parse_dates=['time'])

#inspect it, does it look sane?
ntr_data

In [ ]:
# remove leap days for day of year analysis
ntr_data_noleap = ntr_data[~((ntr_data['time'].dt.month == 2) & (ntr_data['time'].dt.day == 29))].copy()


In [ ]:
ntr_data_noleap

In [ ]:
# for each year, get daily maximum
ntr_data_noleap['year'] = ntr_data_noleap['time'].dt.year
ntr_data_noleap['month'] = ntr_data_noleap['time'].dt.month
ntr_data_noleap['doy'] = ntr_data_noleap['time'].dt.dayofyear
ntr_data_noleap['day'] = ntr_data_noleap['time'].dt.day
ntr_data_noleap['date'] = pd.to_datetime(ntr_data_noleap[['year', 'month', 'day']])


In [ ]:
daily_max = ntr_data_noleap.groupby('date').max().reset_index()
daily_max['year'] = daily_max['date'].dt.year
daily_max['month'] = daily_max['date'].dt.month
daily_max['doy'] = daily_max['date'].dt.dayofyear

In [ ]:
ntr_data_noleap['tide_plus_trend'] = ntr_data_noleap['tide'] + ntr_data_noleap['trend'] 

In [ ]:
def filter_ntr(ntr_data):

    rec_length = (ntr_data['time'].max() - ntr_data['time'].min()).days
    
    
    ntr_decadal, ntr_highFreq = butterworth_lowpass(ntr_data['ntr'], time_diffs, 1/decadal, order=3) 
   
    #interannual
    ntr_interannual, ntr_highFreq = butterworth_lowpass(ntr_highFreq, time_diffs, 1/annual, order=3)

    # intraannual
    ntr_subannual, ntr_highFreq = butterworth_lowpass(ntr_highFreq, time_diffs, 1/monthly, order=5)

    ntr_monthly, ntr_highFreq = butterworth_lowpass(ntr_highFreq, time_diffs, 1/weekly, order=5)

    # Remove high frequencies (weekly to hourly)
    # ntr_weekly is timescales longer than 7 days but less than 1 month
    ntr_weekly, ntr_highFreq = butterworth_lowpass(ntr_highFreq, time_diffs, 1/daily, order=5)

    rank_tide = ntr_data['tide'].rank(method='first', ascending=False)

    # make dataframe of filtered data
    ntr_filtered = pd.DataFrame({'time': ntr_data['time'], 
                             'ntr': ntr_data['ntr'], 
                             'sea_level': ntr_data['sea_level'],
                             'sea_level_detrended': ntr_data['sea_level_detrended'],
                             'tide': ntr_data['tide'],
                             'Trend': ntr_data['trend'],
                            #  'Interdecadal': ntr_interdecadal, 
                             'Decadal': ntr_decadal, 
                             'Interannual': ntr_interannual, 
                             'Seasonal': ntr_data['seasonal_cycle'], 
                             'Subannual': ntr_subannual, 
                             'Monthly': ntr_monthly,
                             'Weekly': ntr_weekly, 
                             'Storms & HF': ntr_highFreq,
                             'Rank Tide': rank_tide } )
                        #    'NTR Trend': ntr_trend_series['ntr']
    

    component_names = list(ntr_filtered.columns) 
    component_names.remove('time')
    component_names.remove('ntr')
    component_names.remove('sea_level')
    component_names.remove('sea_level_detrended')
    component_names.remove('Rank Tide')

    # add trend back into ntr
    # ntr_filtered['ntr'] = ntr_filtered['ntr'] + ntr_filtered['NTR Trend']
    
    return ntr_filtered, component_names


In [ ]:
time_diffs = np.diff(ntr_data['time']).astype('timedelta64[h]').astype(int)

# Define timescales (in days)
annual = 365.25
biannual = 365.25*2
semiannual = 365.25/2
qtrannual = 365.25/4
daily = 7
weekly = 365.25/12
monthly = 3*365.25/12
decadal = 7*365.25

ntr_filtered, component_names = filter_ntr(ntr_data)

In [ ]:
ntr_filtered

In [ ]:
#remove leap days for day of year analysis
ntr_filtered = ntr_filtered[~((ntr_filtered['time'].dt.month == 2) & (ntr_filtered['time'].dt.day == 29))].copy()

ntr_filtered['doy'] = ntr_filtered['time'].dt.dayofyear

ntr_filtered_daily = ntr_filtered.groupby('doy').quantile(0.95).reset_index()
short_term_components = ['Storms & HF', 'Weekly', 'Monthly', 'Subannual']
ntr_filtered_daily['short_term'] = ntr_filtered_daily[short_term_components].sum(axis=1)

# confidence intervals for short term variability
ntr_filtered_daily['short_term_std'] = (ntr_filtered_daily[short_term_components].std(axis=1))
ntr_filtered_daily['short_term_upper'] = (ntr_filtered_daily['short_term'] + 1.96 * ntr_filtered_daily['short_term_std'])
ntr_filtered_daily['short_term_lower'] = (ntr_filtered_daily['short_term'] - 1.96 * ntr_filtered_daily['short_term_std'])




total_components = ['ntr', 'tide']
ntr_filtered_daily['ntr+tide'] = ntr_filtered_daily[total_components].sum(axis=1)
# daily_max_daily = daily_max.groupby('doy').median().reset_index()


# confidence intervals for ntr+tide
ntr_filtered_daily['ntr_std'] = ntr_filtered_daily['ntr+tide'].std(axis=1)
ntr_filtered_daily['ntr+tide_upper'] = (ntr_filtered_daily['ntr+tide'] + 1.96 * ntr_filtered_daily['ntr_std'])
ntr_filtered_daily['ntr+tide_lower'] = (ntr_filtered_daily['ntr+tide'] - 1.96 * ntr_filtered_daily['ntr_std'])

def smooth_30_day(series):
    return series.rolling(window=30, center=True, min_periods=2).mean()


daily_max_smooth = daily_max.copy()
ntr_filtered_daily_smooth = ntr_filtered_daily.copy()
# Apply rolling mean to all relevant columns in one step
columns_to_smooth = ['ntr+tide', 'sea_level', 'tide','ntr+tide_upper', 'ntr+tide_lower']
daily_max_smooth[columns_to_smooth] = daily_max_smooth[columns_to_smooth].rolling(window=30, center=True, min_periods=2).mean()
columns_to_smooth = ['sea_level', 'tide', 'short_term', 'short_term_upper', 'short_term_lower']

ntr_filtered_daily_smooth[columns_to_smooth] = ntr_filtered_daily[columns_to_smooth].rolling(window=30, center=True, min_periods=1).mean()

In [ ]:
# plot ntr_std
daily_max_daily['ntr_std'].plot()
daily_max_daily['ntr+tide'].plot()

In [ ]:
ntr_filtered

In [ ]:
def bootstrap_percentile(data, percentile=95, n_bootstrap=1000, ci=0.95):
    """
    Calculate the confidence interval for a given percentile using bootstrapping.

    Parameters:
        data (array-like): The data to calculate the percentile for.
        percentile (float): The percentile to calculate (default is 95).
        n_bootstrap (int): Number of bootstrap resamples (default is 1000).
        ci (float): Confidence level for the interval (default is 0.95).

    Returns:
        tuple: (percentile_value, lower_bound, upper_bound)
    """
    bootstrapped_percentiles = []
    for _ in range(n_bootstrap):
        resample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_percentiles.append(np.percentile(resample, percentile))
    
    # Calculate the confidence interval bounds
    lower_bound = np.percentile(bootstrapped_percentiles, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_percentiles, (1 + ci) / 2 * 100)
    return np.percentile(data, percentile), lower_bound, upper_bound

In [ ]:
# Group by 'doy' and calculate the 99th percentile with confidence intervals
def calculate_percentile_with_ci(data):
    # expects data to be of the form:
    # data = group['sea_level'].dropna()
    
    if len(data) > 0:
        p99, lower, upper = bootstrap_percentile(data, percentile=99, n_bootstrap=1000, ci=0.95)
        return pd.Series({'sea_level_99th': p99, 'ci_lower': lower, 'ci_upper': upper})
    else:
        return pd.Series({'sea_level_99th': np.nan, 'ci_lower': np.nan, 'ci_upper': np.nan})


In [ ]:
ntr_filtered

In [ ]:
# remove leap days
ntr_filtered = ntr_filtered[~((ntr_filtered['time'].dt.month == 2) & (ntr_filtered['time'].dt.day == 29))].copy()
ntr_filtered['doy'] = ntr_filtered['time'].dt.dayofyear

# # Calculate short_term as the sum of components
short_term_components = ['Storms & HF', 'Weekly', 'Monthly', 'Subannual']
ntr_filtered['short_term'] = ntr_filtered[short_term_components].sum(axis=1)

# group by day
# ntr_filtered_95 = ntr_filtered.groupby('doy').quantile(0.95).reset_index()
# ntr_filtered_max = ntr_filtered.groupby('doy').max().reset_index()

ntr_filtered_daily = ntr_filtered.groupby('doy').agg(
    sea_level_mean=('detrended_sea_level', 'mean'),
    sea_level_std=('detrended_sea_level', 'std'),
    sea_level_95th=('detrended_sea_level', lambda x: np.percentile(x, 95)),
    sea_level_max=('detrended_sea_level', 'max'),
    tide_95th=('tide', lambda x: np.percentile(x, 95)),
    Seasonal=('ntr',lambda x: np.percentile(x, 95)),
    short_term_95th=('short_term', lambda x: np.percentile(x, 95)),
    short_term_std=('short_term', 'std')
).reset_index()

sea_level_daily_99 = ntr_filtered.groupby('doy').apply(calculate_percentile_with_ci).reset_index()
columns_to_smooth = ['sea_level_99th', 'ci_lower', 'ci_upper']
sea_level_daily_99[columns_to_smooth] = sea_level_daily_99[columns_to_smooth].rolling(
    window=30, center=True, min_periods=1
).mean()

# Smooth the daily data with a 30-day rolling mean (like a low-pass filter)
columns_to_smooth = ['detrended_sea_level_max', 'detrended_sea_level_std', 'detrended_sea_level_95th','tide_95th','short_term_95th','short_term_std','Seasonal']
ntr_filtered_daily[columns_to_smooth] = ntr_filtered_daily[columns_to_smooth].rolling(window=30, center=True, min_periods=1).mean()






In [ ]:
msl

In [ ]:
sea_level_daily_99

In [ ]:
top_events

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

# Plot ntr+tide
sns.lineplot(data=sea_level_daily_99-msl, x='doy', y='sea_level_99th', ax=ax, label='total', color='black')
ax.fill_between(
    sea_level_daily_99['doy'],
    sea_level_daily_99['ci_upper']-msl,
    sea_level_daily_99['ci_lower']-msl,
    color='gray',
    alpha=0.2,
    label='total CI'
)

# Plot short_term
sns.lineplot(data=ntr_filtered_daily, x='doy', y='short_term_95th', ax=ax, label='short-term', color='red')
ax.fill_between(
    ntr_filtered_daily['doy'],
    ntr_filtered_daily['short_term_95th'] - ntr_filtered_daily['short_term_std'],
    ntr_filtered_daily['short_term_95th'] + ntr_filtered_daily['short_term_std'],
    color='red',
    alpha=0.2,
    label='short-term CI'
)


sns.lineplot(data=ntr_filtered_daily, x='doy', y='Seasonal', ax=ax, label='seasonal', color='green')
sns.lineplot(data=ntr_filtered_daily, x='doy', y='tide_95th', ax=ax, label='tide (95th percentile)', color='blue')

# sns.lineplot(tp98, ax=ax, label='tide + trend 98th percentile', color='orange')
ax.set_title(f'High water climatology at {station_name} ({station_id})')
ax.set_ylabel('meters')
ax.set_xlabel('Day of Year')

# set xlim to 1-365
ax.set_xlim(1, 365)

# replace numbers with month names
month_starts = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax.set_xticks(month_starts)
ax.set_xticklabels(month_names)

# now add top 10 events at station
top_events = daily_max.nlargest(10, 'sea_level')
msl = ds.MSL.sel(station_id=station_id).item()

top_events['doy'] = top_events['date'].dt.dayofyear

ax.scatter(top_events['doy'], top_events['sea_level']-msl, color='black', zorder=5, label='Top 10 Events')
for i, row in top_events.iterrows():
    ax.text(row['doy'], row['sea_level']-msl+0.03, f"{row['date']:%Y-%m-%d}", color='black', ha='center', rotation=90)

ax.set_title('High Water Climatology')
ax.set_ylabel('Meters, rel. to MSL')
ax.set_xlim(1, 365)
ax.set_ylim(-0.1, 1)
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=3)
plt.show()